In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from coffea import util, hist


In [ ]:
nJets = 4

outputMC = util.load(f'Outputs/outputMCOther_ttgamma_condorFull_{nJets}jet.coffea')
outputMC.add(util.load(f'Outputs/outputMCSingletop_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCTTbar1l_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCTTbar2l_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCTTGamma_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCWJets_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCZJets_ttgamma_condorFull_{nJets}jet.coffea'))

outputData = util.load(f'Outputs/outputData_ttgamma_condorFull_{nJets}jet.coffea')

In [ ]:
grouping= {'ttgamma': ['TTGamma_Dilepton','TTGamma_SingleLept','TTGamma_Hadronic'],
           'ttbar'  : ['TTbarPowheg_Dilepton', 'TTbarPowheg_Semilept', 'TTbarPowheg_Hadronic'],
           'Single top':['ST_s_channel', 'ST_tW_channel', 'ST_tbarW_channel', 'ST_tbar_channel', 'ST_t_channel'],
           'Wgamma' : ['WGamma_01J_5f'],
           'Zgamma' : ['ZGamma_01J_5f_lowMass'],
           'Other'    : ['TTWtoLNu','TTWtoQQ','TTZtoLL','W1jets', 'W2jets', 'W3jets', 'W4jets','DYjetsM10to50', 'DYjetsM50','GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_Pt20to30_Ele', 'QCD_Pt30to50_Ele', 'QCD_Pt50to80_Ele', 'QCD_Pt80to120_Ele', 'QCD_Pt120to170_Ele', 'QCD_Pt170to300_Ele', 'QCD_Pt300toInf_Ele', 'QCD_Pt20to30_Mu', 'QCD_Pt30to50_Mu', 'QCD_Pt50to80_Mu', 'QCD_Pt80to120_Mu', 'QCD_Pt120to170_Mu', 'QCD_Pt170to300_Mu', 'QCD_Pt300to470_Mu', 'QCD_Pt470to600_Mu', 'QCD_Pt600to800_Mu', 'QCD_Pt800to1000_Mu', 'QCD_Pt1000toInf_Mu']
          }

groupingPho= {"Genuine":slice(1,2),
              "MisIDele": slice(2,3),
              "NonPrompt":slice(3,5),
             }


In [ ]:
MisIDSFResults = {'FSRDown': 2.3258888950981658,
 'FSRUp': 2.323321648572707,
 'ISRDown': 2.3230711845246037,
 'ISRUp': 2.3250742760361485,
 'JERDown': 2.4916358829485334,
 'JERUp': 2.1624914842453293,
 'JESDown': 2.509294663019229,
 'JESUp': 2.135357294316598,
 'PDFDown': 2.324502013792059,
 'PDFUp': 2.323545650509748,
 'Q2ScaleDown': 2.3293301438482605,
 'Q2ScaleUp': 2.3167686733720436,
 'btagWeight_heavyDown': 2.3161258587737783,
 'btagWeight_heavyUp': 2.3318748601639374,
 'btagWeight_lightDown': 2.314511973536161,
 'btagWeight_lightUp': 2.33352399467015,
 'eleEffWeightDown': 2.3554278067205736,
 'eleEffWeightUp': 2.2932330953488718,
 'muEffWeightDown': 2.3240404881606733,
 'muEffWeightUp': 2.3240404881606733,
 'nominal': 2.3240404882109207,
 'puWeightDown': 2.5637652930311705,
 'puWeightUp': 2.1120559976304656}


In [ ]:
photonPurityResults = {'FSRDown': (0.8030142091355378, 0.02656639631207204),
 'FSRUp': (0.803467537651622, 0.02629404549365212),
 'ISRDown': (0.8018302949232643, 0.026126375541722265),
 'ISRUp': (0.804461717954526, 0.026444066165957382),
 'JERDown': (0.8051019088926759, 0.026506921728798816),
 'JERUp': (0.7984673072982711, 0.026222706737238234),
 'JESDown': (0.8060810360464794, 0.026711459939952456),
 'JESUp': (0.8002546049275313, 0.026059478589156537),
 'PDFDown': (0.8035204920374956, 0.02653775791567313),
 'PDFUp': (0.8035307005167678, 0.026281259025561554),
 'Q2ScaleDown': (0.8043317356519103, 0.027877124454999947),
 'Q2ScaleUp': (0.8020354226529646, 0.024999047555065074),
 'btagWeight_heavyDown': (0.8032875411324775, 0.026463296023541468),
 'btagWeight_heavyUp': (0.8035893554441516, 0.026337141144431832),
 'btagWeight_lightDown': (0.8034995393387975, 0.02643523179281183),
 'btagWeight_lightUp': (0.8033743258917129, 0.02636304397032702),
 'eleEffWeightDown': (0.803453136283781, 0.026436277365020047),
 'eleEffWeightUp': (0.8034205363035585, 0.026361799486459368),
 'muEffWeightDown': (0.8034257479581941, 0.026434470425939233),
 'muEffWeightUp': (0.8034475206831849, 0.026363581517734703),
 'nominal': (0.8034366347463864, 0.02639897109376131),
 'puWeightDown': (0.8079974209720092, 0.026502251190634044),
 'puWeightUp': (0.798484309778782, 0.026287794984425114)}



In [ ]:
topPurityResults = {'FSRDown': (0.7625088925186997, 0.054231604484011736),
 'FSRUp': (0.7696066723943817, 0.05814402063353581),
 'ISRDown': (0.822724047640092, 0.05839679242422305),
 'ISRUp': (0.7238656927125936, 0.05548770340953093),
 'JERDown': (0.7693957648855223, 0.05317617974697675),
 'JERUp': (0.7478146052911734, 0.05722566554922455),
 'JESDown': (0.7945911123489572, 0.054569995530775676),
 'JESUp': (0.7141144449951271, 0.057289368764441906),
 'PDFDown': (0.7657202166884056, 0.05667093455148253),
 'PDFUp': (0.7662539910761348, 0.0562304496271842),
 'Q2ScaleDown': (0.7597117742552794, 0.0593452338480461),
 'Q2ScaleUp': (0.7762575130283725, 0.053551073218728366),
 'btagWeight_heavyDown': (0.7644770428104973, 0.056805065916077514),
 'btagWeight_heavyUp': (0.7665034940652995, 0.05609976526397491),
 'btagWeight_lightDown': (0.7630598879751872, 0.05654260955900684),
 'btagWeight_lightUp': (0.7681546197410744, 0.056333544349048054),
 'eleEffWeightDown': (0.7651281095045667, 0.05656284797091966),
 'eleEffWeightUp': (0.7658846670114646, 0.05633446162264847),
 'muEffWeightDown': (0.7652624114194411, 0.05653088379696748),
 'muEffWeightUp': (0.7657516385354254, 0.056366155471439895),
 'nominal': (0.7655058238808953, 0.05644847783453547),
 'puWeightDown': (0.7559069098357679, 0.05526018303340832),
 'puWeightUp': (0.7783595397212802, 0.05774640772959691)}



In [ ]:
h = outputMC['M3'].sum('lepFlavor').sum('M3').group('dataset',hist.Cat(r'dataset',r'Samples',sorting='placement'),grouping)
h = h.group('category',hist.Cat(r'category',r'Category',sorting='placement'),groupingPho)


In [ ]:
hData = outputData['M3'].sum('lepFlavor').sum('M3').sum('dataset').sum('category')
nData = hData.values()[('noweight',)]


In [ ]:
def getPseudoData(mcYield, mcYieldErr, ttgammaSF=1., npSF=1., asimov=False):
    mcYieldTest = mcYield.copy()
    mcYieldTest[0] *=  ttgammaSF
    mcYieldTest[:,2] *= npSF
    
    if not asimov:
        mcYieldTest = np.random.poisson(mcYieldTest)

    photonPurity = mcYieldTest[:,:2].sum()/mcYieldTest.sum()
    photonPurityErr = 0.04

    topPurity = mcYieldTest[:2].sum()/mcYieldTest.sum()
    topPurityErr = 0.02

    nData = mcYieldTest.sum()

    return photonPurity, photonPurityErr, topPurity, topPurityErr, nData


In [ ]:
#import numba

def likelihoodFunction(ttgammaSF=1., nonPromptSF=1., mcValues=None, mcValuesErr=None, photonPurity=None, topPurity=None, nData = None, photonPurityErr=None, topPurityErr=None):
    mcValues[0] *= ttgammaSF
    mcValuesErr[0] *= ttgammaSF

    mcValues[:,-1] *= nonPromptSF
    mcValuesErr[:,-1] *= nonPromptSF
    
    nMC = mcValues.sum()
    nMCErr = (mcValuesErr**2).sum()**0.5

    #nIso is the sum of the first two columsn of data (genuine and misID)
    nIso = mcValues[:,0:2].sum()
    nIsoErr = (mcValuesErr[:,0:2]**2).sum()**0.5

    #nTop is the sum of the first two rows (ttgamma and ttbar)
    nTop = mcValues[0:2].sum()
    nTopErr = (mcValuesErr[0:2]**2).sum()**0.5

    mcPhotonPurity    = nIso/nMC
    mcPhotonPurityErr = (mcPhotonPurity) * ((nIsoErr/nIso)**2 + (nMCErr/nMC)**2)**0.5

    mcTopPurity = nTop/nMC
    mcTopPurityErr = mcTopPurity * ((nTopErr/nTop)**2 + (nMCErr/nMC)**2)**0.5
    
    
    chi2 = ((photonPurity-mcPhotonPurity)**2/(photonPurityErr**2 + mcPhotonPurityErr**2) + 
            (topPurity - mcTopPurity)**2/(topPurityErr**2 + mcTopPurityErr**2) +
            (nData - nMC)**2/(nData + nMCErr**2)
           )
    
    return np.exp(-0.5*chi2)
                
    

In [ ]:
def maximizeLikelihood(fitData, 
                       ttgSF = None, nonPromptSF=None, 
                       startTTGamma = 1., startNonprompt = 1., 
                       nSteps = 100,
                       verbose=False, 
                       find1sigma = False, 
                       nStepsErr = 20):


    mcYield=fitData['mcYield']
    mcYieldErr=fitData['mcYieldErr']
    photonPurity=fitData['photonPurity']
    photonPurityErr=fitData['photonPurityErr']
    topPurity=fitData['topPurity']
    topPurityErr=fitData['topPurityErr']
    nData=fitData['nData']
    
    
    if ttgSF is None:
        ttgSF=startTTGamma
        iTTGSteps = range(3)
    else:
        iTTGSteps = [1]

    if nonPromptSF is None:
        nonPromptSF=startNonprompt
        iNPSteps = range(3)
    else:
        iNPSteps = [1]
        
    stepSize = 0.1
    
    lastStepLk = -1.

    for steps in range(nSteps):

        best_lk = -1
        best_iTTG = -1
        best_iNP = -1
        
        for iTTG in iTTGSteps:
            for iNP in iNPSteps:
                lk = likelihoodFunction(ttgammaSF = ttgSF + (iTTG-1)*stepSize , 
                                        nonPromptSF=nonPromptSF + (iNP-1)*stepSize, 
                                        mcValues=mcYield.copy(), mcValuesErr=mcYieldErr.copy(),
                                        photonPurity=photonPurity, topPurity=topPurity, 
                                        photonPurityErr=photonPurityErr, topPurityErr=topPurityErr, 
                                        nData=nData)

                if lk > best_lk:
                    best_lk = lk
                    best_iTTG = iTTG
                    best_iNP = iNP

        ttgSF = ttgSF+(best_iTTG-1)*stepSize
        nonPromptSF = nonPromptSF + (best_iNP-1)*stepSize

        if best_iTTG==best_iNP==1:
            stepSize = stepSize/2.
            lastStepLk=-1.

        if verbose:
            print(steps, ttgSF, nonPromptSF, stepSize, best_lk)
            
    if find1sigma:

        ### scan for ttgSF down
        ttgammaErr = 0.1
        stepSize = 0.1
        trend = 1
        for step in range(nStepsErr):
            _,_,_lk = maximizeLikelihood(fitData, 
                                         ttgSF = ttgSF - ttgammaErr, 
                                         nonPromptSF=None, 
                                         nSteps = 50,
                                    )


            NLL = -2*np.log(_lk/best_lk)

            if NLL < 1:
                if trend== -1:
                    stepSize /= 2.
                ttgammaErr += stepSize
                trend=1
            else:
                if trend==1:
                    stepSize /= 2.
                ttgammaErr -= stepSize
                trend=-1
        ttgammaDown = -1*ttgammaErr

        ### scan for ttgSF up
        ttgammaErr = 0.1
        stepSize = 0.1
        trend = 1
        for step in range(nStepsErr):
            _,_,_lk = maximizeLikelihood(fitData, 
                                         ttgSF = ttgSF + ttgammaErr, 
                                         nonPromptSF=None, 
                                         nSteps = 50,
                                        )

            NLL = -2*np.log(_lk/best_lk)

            if NLL < 1:
                if trend== -1:
                    stepSize /= 2.
                ttgammaErr += stepSize
                trend=1
            else:
                if trend==1:
                    stepSize /= 2.
                ttgammaErr -= stepSize
                trend=-1
        ttgammaUp = ttgammaErr

        ### scan for npSF down
        npErr = 0.1
        stepSize = 0.1
        trend = 1
        for step in range(nStepsErr):
            _,_,_lk = maximizeLikelihood(fitData, 
                                         ttgSF = None, 
                                         nonPromptSF=nonPromptSF - npErr, 
                                         nSteps = 50,
                                        )            

            NLL = -2*np.log(_lk/best_lk)

            if NLL < 1:
                if trend== -1:
                    stepSize /= 2.
                npErr += stepSize
                trend=1
            else:
                if trend==1:
                    stepSize /= 2.
                npErr -= stepSize
                trend=-1
        nonPromptDown = -1*npErr

        ### scan for npSF up        
        npErr = 0.1
        stepSize = 0.1
        trend = 1
        for step in range(nStepsErr):
            _,_,_lk = maximizeLikelihood(fitData, 
                                         ttgSF = None, 
                                         nonPromptSF=nonPromptSF + npErr,
                                         nSteps = 50,                                         
                                        )            

            NLL = -2*np.log(_lk/best_lk)

            if NLL < 1:
                if trend== -1:
                    stepSize /= 2.
                npErr += stepSize
                trend=1
            else:
                if trend==1:
                    stepSize /= 2.
                npErr -= stepSize
                trend=-1
        nonPromptUp = npErr        
        
        return ttgSF, ttgammaUp, ttgammaDown, nonPromptSF, nonPromptUp, nonPromptDown, best_lk
        
    else:
        return ttgSF, nonPromptSF, best_lk


In [ ]:
vals = h.integrate('systematic','nominal').values()
mcYield = []

for i, sample in enumerate(grouping.keys()):
    mcYield.append([])
    for j, category in enumerate(groupingPho.keys()):
        v = vals[(category,sample)]
        mcYield[-1].append(v)
mcYield = np.array(mcYield)

errs = h.integrate('systematic','nominal')._sumw2

mcYieldErr = []
for s in errs:
    mcYieldErr.append(errs[s]**0.5)

mcYieldErr = np.array(mcYieldErr)
mcYieldErr.shape = (3,6)
mcYieldErr = mcYieldErr.transpose()

misIDEleSF = MisIDSFResults['nominal']

mcYield[:,1] *= misIDEleSF
mcYieldErr[:,1] *= misIDEleSF


fitData = {
    'mcYield':mcYield, 
    'mcYieldErr':mcYieldErr,
    'photonPurity':photonPurityResults['nominal'][0], 
    'photonPurityErr':photonPurityResults['nominal'][1],
    'topPurity':topPurityResults['nominal'][0],
    'topPurityErr':topPurityResults['nominal'][1], 
    'nData':nData, 
}

output = maximizeLikelihood(fitData,
                        nSteps=100, 
                        find1sigma=True, 
                        nStepsErr=100, 
                        startTTGamma=1,
                        startNonprompt=1
                       )
bestTTGSF, bestTTGSF_Up, bestTTGSF_Down, bestNPSF, bestNPSF_Up, bestNPSF_Down, mxLk = output                                                                                                         
print("TTGamma SF = %.4f +%.4f %.4f"%(bestTTGSF, bestTTGSF_Up, bestTTGSF_Down))
print("nonPrompt SF = %.4f +%.4f %.4f"%(bestNPSF, bestNPSF_Up, bestNPSF_Down))


In [ ]:
systematics = list(photonPurityResults.keys())
systematics.remove('nominal')

systResults = {}

for syst in systematics:
    vals = h.integrate('systematic',syst).values()
    mcYield = []

    for i, sample in enumerate(grouping.keys()):
        mcYield.append([])
        for j, category in enumerate(groupingPho.keys()):
            v = vals[(category,sample)]
            mcYield[-1].append(v)
    mcYield = np.array(mcYield)

    errs = h.integrate('systematic',syst)._sumw2

    mcYieldErr = []
    for s in errs:
        mcYieldErr.append(errs[s]**0.5)

    mcYieldErr = np.array(mcYieldErr)
    mcYieldErr.shape = (3,6)
    mcYieldErr = mcYieldErr.transpose()

    misIDEleSF = MisIDSFResults[syst]

    mcYield[:,1] *= misIDEleSF
    mcYieldErr[:,1] *= misIDEleSF

    
    fitDataSyst = {
        'mcYield':mcYield, 
        'mcYieldErr':mcYieldErr,
        'photonPurity':photonPurityResults[syst][0], 
        'photonPurityErr':photonPurityResults[syst][1],
        'topPurity':topPurityResults[syst][0],
        'topPurityErr':topPurityResults[syst][1], 
        'nData':nData, 
    }
    
    output = maximizeLikelihood(fitDataSyst,
                            nSteps=100, 
                            find1sigma=False, 
                            nStepsErr=100, 
                            startTTGamma=1,
                            startNonprompt=1
                           )
    bestTTGSF, bestNPSF, bestLk = output
    systResults[syst] = bestTTGSF

print (systResults)

In [ ]:
lkVals = []
ttgVals = np.arange(0,2.,.01)
for ttgSF in ttgVals:
    _ttg, _np, _lk = maximizeLikelihood(fitData, ttgSF = ttgSF, nSteps=50)
    lkVals.append(-2*np.log(_lk/mxLk))

lkVals = np.array(lkVals)

plt.scatter(ttgVals[lkVals<6],lkVals[lkVals<6],color='blue')
plt.scatter(ttgVals[lkVals<4],lkVals[lkVals<4],color='yellow')
plt.scatter(ttgVals[lkVals<1],lkVals[lkVals<1],color='green')
plt.ylabel("2*NLL")
plt.xlabel("$t\overline{t}\gamma$ SF")

ttgVals2Sig = ttgVals[lkVals<5]




In [ ]:
lkVals = []
npVals = np.arange(0,2.,.01)
for npSF in npVals:
    _ttg, _np, _lk = maximizeLikelihood(fitData, nonPromptSF= npSF, nSteps=50)
    lkVals.append(-2*np.log(_lk/mxLk))

lkVals = np.array(lkVals)

plt.scatter(npVals[lkVals<6],lkVals[lkVals<6],color='blue')
plt.scatter(npVals[lkVals<4],lkVals[lkVals<4],color='yellow')
plt.scatter(npVals[lkVals<1],lkVals[lkVals<1],color='green')
plt.ylabel("2*NLL")
plt.xlabel("Nonprompt SF")

npVals2Sig = npVals[lkVals<5]




In [ ]:
lkVals = []
for npSF in npVals2Sig:
    _lkVals = []
    for ttg in ttgVals2Sig:
        _ttg, _np, _lk = maximizeLikelihood(fitData, ttgSF= ttg, nonPromptSF= npSF, nSteps=1)
        _lkVals.append(-2*np.log(_lk/mxLk))
    lkVals.append(_lkVals)
lkVals = np.array(lkVals)

In [ ]:
plt.contourf(ttgVals2Sig, npVals2Sig, lkVals, [0,1,4],colors=['green','yellow'])
plt.plot(bestTTGSF, bestNPSF,marker='+',color='black',markersize=12,markeredgewidth=3);
